Extrae el % de bateria del comentario y lo pega en la columna de bateria

In [4]:
import os
import pandas as pd
import re
from datetime import datetime

In [5]:
elemento_a_buscar = "iphone"
fecha = datetime.now().strftime("%Y%m%d")
carpeta_origen = "../data/4_download_description"
carpeta_destino = "../data/5_data_from_comments"

In [6]:
def encontrar_bateria(texto: str):
    # Verificar si el texto es NaN o similar
    if pd.isna(texto):
        return None
    # Buscar números seguidos por %
    numeros_con_porcentaje = re.findall(r'\b(8[0-9]|9[0-9]|100)\b%', texto)
    # Si encuentra números con %, devolver el primero
    if numeros_con_porcentaje:
        return numeros_con_porcentaje[0] 
    # Si no hay % buscar solo los números
    numeros = re.findall(r'\b(8[0-9]|9[0-9]|100)\b', texto)
    # Si encuentra números, devolver el primero
    if numeros:
        return numeros[0]
    # Si no encuentra ningún número, devolver None
    return None
# buscar 128|256|512 seguido de \s*(Mb|MB|mb)
# si no, buscar \b1[Tt]\b
# si no, buscar \b(128|256|512)\b
def encontrar_memoria(texto: str) -> str:
    if pd.isna(texto):
        return None
    texto = str(texto)
    
    # Buscar 128|256|512 seguido de \s*(Mb|MB|mb)
    memorias_con_unidades = re.search(r'\b(128|256|512)\b\s*(Mb|MB|mb)', texto)
    if memorias_con_unidades:
        memoria, unidad = memorias_con_unidades.groups()
        return memoria + unidad
    
    # Buscar \b1[Tt]\b
    memoria_1t = re.search(r'\b1[Tt]', texto)
    if memoria_1t:
        return '_1T'
    
    # Buscar \b(128|256|512)\b
    memorias_sin_unidades = re.search(r'\b(128|256|512)\b', texto)
    if memorias_sin_unidades:
        return memorias_sin_unidades.group(0)
    
    return None
    
def es_tienda(texto: str) -> bool:
    if pd.isna(texto):
        return None
    words_to_find = ('reacondicionado', 'tienda')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

def tiene_garantia(texto: str):
    if pd.isna(texto):
        return None
    words_to_find = ('garantia', 'care', 'factura')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

In [10]:
def get_data_from_comments(n):
    n=str(n)
    filepath = os.path.join(carpeta_origen, f"{elemento_a_buscar}_{fecha}_{n}.csv")
    # Si el archivo no existe para
    if not os.path.exists(filepath):
        return None
    df_original = pd.read_csv(filepath)
    df_original.drop('Unnamed: 0', axis=1, inplace=True)
    #df_original.head(2)
    df = df_original.copy()
    missing_values = df_original.isnull().sum()
    print('missing_values', missing_values)
    df['bateria'] = df_original['comentario'].apply(encontrar_bateria)
    df['memoria'] = df_original['comentario'].apply(encontrar_memoria)
    df['tienda'] = df_original['comentario'].apply(es_tienda)
    df['garantia'] = df_original['comentario'].apply(tiene_garantia)
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    df.to_csv(os.path.join(carpeta_destino, f"{fecha}_{elemento_a_buscar}_{n}.csv"))

    missing_values = df_original.isnull().sum()
    print('df_original', missing_values)
    missing_values = df.isnull().sum()
    print("df missing values: ", missing_values)

In [11]:
for n in range(10):
    get_data_from_comments(n)

missing_values id                0
time_scrap        0
gen               0
mod               0
memoria           3
bateria           3
precio            0
estado            0
nombre            0
tiene_color       0
tiene_emojis      0
tiene_revisado    0
municipio         0
distancia         0
reservado         0
url_articulo      0
fecha             0
comentario        0
dtype: int64
df_original id                0
time_scrap        0
gen               0
mod               0
memoria           3
bateria           3
precio            0
estado            0
nombre            0
tiene_color       0
tiene_emojis      0
tiene_revisado    0
municipio         0
distancia         0
reservado         0
url_articulo      0
fecha             0
comentario        0
dtype: int64
df missing values:  id                0
time_scrap        0
gen               0
mod               0
memoria           0
bateria           2
precio            0
estado            0
nombre            0
tiene_color       0
tiene_e